In [1]:
output_folder = 'avgint'
output_filename = 'avgi'
crop_margins_ratio = [0.2, 0.8, 0.2, 0.6]  # minX, maxX, minY, maxY

from local_vars import root_folder
import os

output_fullpath = os.path.join(root_folder, output_folder)
output_fullname = os.path.join(output_fullpath, output_filename)

if not os.path.exists(output_fullpath):
    os.makedirs(output_fullpath)

In [2]:
sequence_browser_node = slicer.util.getFirstNodeByName('', className='vtkMRMLSequenceBrowserNode')
print "Sequence browser node: " + str(sequence_browser_node.GetID())

n = sequence_browser_node.GetNumberOfItems()
sequence_browser_node.SelectFirstItem()

print "Number of images: " + str(n)

Sequence browser node: vtkMRMLSequenceBrowserNode1
Number of images: 3737


In [4]:
master_node = sequence_browser_node.GetMasterSequenceNode()
image_node = sequence_browser_node.GetProxyNode(master_node)
image_data = image_node.GetImageData()

dims = image_data.GetDimensions()
crop_px = [int(dims[0]*crop_margins_ratio[0]),
           int(dims[0]*crop_margins_ratio[1]),
           int(dims[1]*crop_margins_ratio[2]),
           int(dims[1]*crop_margins_ratio[3])]

print "Ultrasound image node: " + image_node.GetName()
print "- dimensions: " + str(dims)
print "- crop margins: " + str(crop_px)

Ultrasound image node: Image_Image
- dimensions: (512, 512, 1)
- crop margins: [102, 409, 102, 307]


In [26]:
import numpy as np

image_extent = image_data.GetExtent()
avg_intensities = np.zeros([n])

for i in range(n):
    image_data = image_node.GetImageData()
    
    image_array = slicer.util.arrayFromVolume(image_node)
    cropped_array = image_array[:, crop_px[0]:crop_px[1], crop_px[2]:crop_px[3]]
    avg_intensity = np.average(cropped_array)
    avg_intensities[i] = avg_intensity
    
    # slicer.app.processEvents()
    sequence_browser_node.SelectNextItem()
    
print "Computed averages"

Computed averages


In [27]:
timestamps = np.zeros([n])

for i in range(n):
    timestamps[i] = float(master_node.GetNthIndexValue(i))
        

In [28]:
slicer.app.layoutManager().setLayout(slicer.vtkMRMLLayoutNode.SlicerLayoutConventionalPlotView)

table_node = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLTableNode')
table = table_node.GetTable()

arr_x = vtk.vtkFloatArray()
arr_x.SetName('Time')
table.AddColumn(arr_x)

arr_y = vtk.vtkFloatArray()
arr_y.SetName('Avg intensity')
table.AddColumn(arr_y)

table.SetNumberOfRows(n)
for i in range(n):
    table.SetValue(i, 0, timestamps[i])
    table.SetValue(i, 1, avg_intensities[i])

plot_series_node = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLPlotSeriesNode', "Avg intensity")
plot_series_node.SetAndObserveTableNodeID(table_node.GetID())
plot_series_node.SetXColumnName("Time")
plot_series_node.SetYColumnName("Avg intensity")
plot_series_node.SetPlotType(slicer.vtkMRMLPlotSeriesNode.PlotTypeScatter)
plot_series_node.SetMarkerStyle(slicer.vtkMRMLPlotSeriesNode.MarkerStyleNone)
plot_series_node.SetUniqueColor()

plot_chart_node = slicer.mrmlScene.AddNewNodeByClass('vtkMRMLPlotChartNode')
plot_chart_node.AddAndObservePlotSeriesNodeID(plot_series_node.GetID())
plot_chart_node.SetTitle('Average intensities for each frame')
plot_chart_node.SetXAxisTitle('Time (s)')
plot_chart_node.SetYAxisTitle('Average intensity (0..255)')

layout_manager = slicer.app.layoutManager()
plot_layout = slicer.modules.plots.logic().GetLayoutWithPlot(layout_manager.layout)
layout_manager.setLayout(plot_layout)

plot_widget = layout_manager.plotWidget(0)
plot_view_node = plot_widget.mrmlPlotViewNode()
plot_view_node.SetPlotChartNodeID(plot_chart_node.GetID())

In [29]:
# np.save(output_fullname, avg_intensities)

print (avg_intensities[:4])

[ 82.11858266  82.11858266  82.16408993  82.21981409]
